In [ ]:
import requests
import numpy as np
import pandas as pd
from datetime import datetime
from tqdm import tqdm

In [2]:
years = [y for y in range(2006, 2026)]
for year in years:
    url = f'https://www.fire.ca.gov/api/sitecore/Incident/GetFiresByYear?year={year}'
    print(url)

https://www.fire.ca.gov/api/sitecore/Incident/GetFiresByYear?year=2006
https://www.fire.ca.gov/api/sitecore/Incident/GetFiresByYear?year=2007
https://www.fire.ca.gov/api/sitecore/Incident/GetFiresByYear?year=2008
https://www.fire.ca.gov/api/sitecore/Incident/GetFiresByYear?year=2009
https://www.fire.ca.gov/api/sitecore/Incident/GetFiresByYear?year=2010
https://www.fire.ca.gov/api/sitecore/Incident/GetFiresByYear?year=2011
https://www.fire.ca.gov/api/sitecore/Incident/GetFiresByYear?year=2012
https://www.fire.ca.gov/api/sitecore/Incident/GetFiresByYear?year=2013
https://www.fire.ca.gov/api/sitecore/Incident/GetFiresByYear?year=2014
https://www.fire.ca.gov/api/sitecore/Incident/GetFiresByYear?year=2015
https://www.fire.ca.gov/api/sitecore/Incident/GetFiresByYear?year=2016
https://www.fire.ca.gov/api/sitecore/Incident/GetFiresByYear?year=2017
https://www.fire.ca.gov/api/sitecore/Incident/GetFiresByYear?year=2018
https://www.fire.ca.gov/api/sitecore/Incident/GetFiresByYear?year=2019
https:

miliseconds -> timenow

In [ ]:
from datetime import datetime

dt = datetime.fromtimestamp(1139200200000 // 1000)
dt

datetime.datetime(2006, 2, 6, 13, 30)

In [43]:
url = 'https://www.fire.ca.gov/api/sitecore/Incident/GetFiresByYear?year=2023'

params = {"year": 2023}
headers = {"User-Agent": "Mozilla/5.0"}  # 해외 IP 차단 우회에 도움될 수도 있음

response = requests.get(url, params=params, headers=headers)
print(response.status_code)
print(response.text)

403
<HTML><HEAD>
<TITLE>Access Denied</TITLE>
</HEAD><BODY>
<H1>Access Denied</H1>
 
You don't have permission to access "http&#58;&#47;&#47;www&#46;fire&#46;ca&#46;gov&#47;api&#47;sitecore&#47;Incident&#47;GetFiresByYear&#63;" on this server.<P>
Reference&#32;&#35;18&#46;2dad2c17&#46;1756304453&#46;16aec81a
<P>https&#58;&#47;&#47;errors&#46;edgesuite&#46;net&#47;18&#46;2dad2c17&#46;1756304453&#46;16aec81a</P>
</BODY>
</HTML>



## API로 데이터 불러오기

내 코드

In [ ]:
response = requests.get(url)
fire = pd.DataFrame()
years = [year for year in range(2006, 2026)]
headers = {"User-Agent": "Mozilla/5.0"}

for year in tqdm(years):
    url = f'https://www.fire.ca.gov/api/sitecore/Incident/GetFiresByYear?year={year}'
    response = requests.get(url, headers=headers)
    data = response.json()
    cols = data[0].keys()
    data_array = np.array([[value for value in data[i].values()] for i in range(len(data))])
    if year == years[0]:
        fire = pd.DataFrame(data_array, columns=cols)
    else:
        new = pd.DataFrame(data_array, columns=cols)
        fire = pd.concat([fire, new], ignore_index=True)
fire

In [ ]:
headers = {"User-Agent": "Mozilla/5.0"}

#singleFire = pd.DataFrame()
for url in tqdm(fire['Url'][697:]):
    url2 = f'https://www.fire.ca.gov/api/sitecore/Incident/GetSingleFire?IncidentUrl={url}'
    response = requests.get(url2, headers=headers)
    data = response.json()
    cols = data.keys()
    data_array = [[value for value in data.values()]]
    new = pd.DataFrame(data_array, columns=cols)
    singleFire = pd.concat([singleFire, new], ignore_index=True)

개선된 코드 (gpt)

In [ ]:
import requests
import pandas as pd

years = range(2006, 2026)
headers = {"User-Agent": "Mozilla/5.0"}

all_data = []  # 데이터를 모아두는 리스트

for year in years:
    url = f'https://www.fire.ca.gov/api/sitecore/Incident/GetFiresByYear?year={year}'
    response = requests.get(url, headers=headers)

    if response.status_code != 200:
        print(f"⚠️ {year} 요청 실패: {response.status_code}")
        continue

    try:
        data = response.json()
        if not data:  # 데이터가 비어있는 경우
            print(f"ℹ️ {year} 데이터 없음")
            continue

        df = pd.DataFrame(data)
        df["year"] = year  # 출처 구분을 위해 연도 컬럼 추가
        all_data.append(df)

    except ValueError:
        print(f"⚠️ {year} JSON 파싱 실패")
        print(response.text[:200])  # 앞부분만 확인

# 모든 연도 데이터 합치기
if all_data:
    fire = pd.concat(all_data, ignore_index=True)
    print(f"✅ 총 {len(fire)}건 데이터 수집 완료")
else:
    fire = pd.DataFrame()
    print("❌ 수집된 데이터 없음")

fire.head()

In [ ]:
headers = {"User-Agent": "Mozilla/5.0"}

all_fires = []

for url in tqdm(fire['Url'], desc="Fetching fire details"):
    url2 = f'https://www.fire.ca.gov/api/sitecore/Incident/GetSingleFire?IncidentUrl={url}'
    response = requests.get(url2, headers=headers)

    if response.status_code != 200:
        print(f"⚠️ 요청 실패 ({response.status_code}) - {url}")
        continue

    try:
        data = response.json()
        if not data:
            print(f"ℹ️ 데이터 없음 - {url}")
            continue

        # JSON dict → DataFrame 한 줄
        df = pd.DataFrame([data])
        all_fires.append(df)

    except ValueError:
        print(f"⚠️ JSON 파싱 실패 - {url}")
        print(response.text[:200])  # 앞부분만 확인

# 모든 데이터 합치기
if all_fires:
    singleFire = pd.concat(all_fires, ignore_index=True)
    print(f"✅ 총 {len(singleFire)} 건의 상세 데이터 수집 완료")
else:
    singleFire = pd.DataFrame()
    print("❌ 수집된 데이터 없음")

singleFire.head()

## Start

In [125]:
# fire = pd.read_excel(r'D:\vscode\RhombusPower\fire.xlsx')
# singleFire = pd.read_excel(r'D:\vscode\RhombusPower\singleFire.xlsx')

fire = pd.read_excel(r'C:\Users\SSAFY\vscode_psb\liveCodingTest\fire.xlsx')
singleFire = pd.read_excel(r'C:\Users\SSAFY\vscode_psb\liveCodingTest\singleFire.xlsx')

display(fire)
display(singleFire)

,Name,Updated,Started,AdminUnit,County,Location,AcresBurned,PercentContained,Longitude,Latitude,Type,UniqueId,Url,StartedDateOnly,IsActive,ExternalUrl
0,Sierra Fire,2006-02-12,2006-02-06,USFS/Orange County Fire,"Orange,Riverside",Corona/Orange,10584.0,100,0.000000,0.000000,NaN,7bc3e973-7c13-4e62-926a-c6d25d4fa426,/incidents/2006/2/6/sierra-fire/,NaN,False,NaN
1,Arrastre Fire,2006-06-09,2006-06-09,San Bernardino National Forest,San Bernardino,6 miles east of Big Bear City,361.0,100,0.000000,0.000000,NaN,8e7f4ac2-1628-4263-97d9-3c8d9da126e3,/incidents/2006/6/9/arrastre-fire/,NaN,False,NaN
2,Plunge Fire,2006-06-19,2006-01-23,San Bernardino National Forest,San Bernardino,2-3 miles south of Running Spring,485.0,100,0.000000,0.000000,NaN,16d75336-a2e0-4dc0-b894-873e80e3db9f,/incidents/2006/1/23/plunge-fire/,NaN,False,NaN
3,Perkins Fire,2006-06-25,2006-06-19,Los Padres National Forest,Santa Barbara,South of the community of New,15043.0,100,0.000000,0.000000,NaN,9f2d8497-ffff-4d8e-a0ac-38a340fef8ab,/incidents/2006/6/19/perkins-fire/,NaN,False,NaN
4,Del Puerto Fire,2003-07-24,2003-07-20,CDF Santa Clara Unit,Stanislaus,Del Puerto Canyon Road/MP 16,5909.0,100,0.000000,0.000000,NaN,472b6bb1-1cf3-4157-810b-6f298ba3f938,/incidents/2006/7/20/del-puerto-fire/,NaN,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4008,Butter Fire,2025-08-26,2025-08-26,Bureau of Land Management - California Desert ...,Kern,"Butterbredt Canyon Road, East of Pacific Crest...",48.0,0,-118.147758,35.464035,Wildfire,aa24ecdd-5b7c-4ef2-8423-dc88687c5965,/incidents/2025/8/26/butter-fire/,NaN,True,NaN
4009,Grade Fire,2025-08-27,2025-08-26,CAL FIRE Siskiyou Unit,Siskiyou,"West Lava Rim Road, Pondosa",10.0,15,-121.590689,41.217878,Wildfire,eb382b3b-d965-4bae-a906-c1bde34d4ec5,/incidents/2025/8/26/grade-fire/,NaN,True,NaN
4010,3-27 Fire,2025-08-26,2025-08-26,CAL FIRE Lassen-Modoc Unit,Lassen,"Southeast of Pumpkin Center, Bieber",24.0,0,-121.057196,41.088854,Wildfire,270ab2cc-1c68-4ffc-a7b2-5990bebe1de6,/incidents/2025/8/26/3-27-fire/,NaN,True,NaN
4011,H-10 Fire,2025-08-26,2025-08-26,Lassen National Forest,Shasta,"Forest Road 36N18 & Butte Creek, Hat Creek",10.6,0,-121.330697,40.737144,Wildfire,48a9cb94-d7ee-482e-a4fe-274f5b20d1ac,/incidents/2025/8/26/h-10-fire/,NaN,True,NaN


,Name,Final,Updated,Started,AdminUnit,AdminUnitUrl,County,Counties,Location,AcresBurned,...,Agencies,UnifiedCommandAgencies,mapZoom,AdditionalInfo,IncidentCameras,CameraString,Photos,MapUrl3D,DemographicsReportUrl,AdditionalIncidentMapsUrl
0,Sierra Fire,True,/Date(1139767200000)/,/Date(1139200200000)/,USFS/Orange County Fire,NaN,NaN,"['Orange', 'Riverside']",Corona/Orange,10584.0,...,"[{'Name': 'USFS/Orange County Fire', 'AgencyUR...",[],12,[],NaN,NaN,[],NaN,NaN,NaN
1,Arrastre Fire,True,/Date(1149867180000)/,/Date(1149857460000)/,San Bernardino National Forest,NaN,NaN,['San Bernardino'],6 miles east of Big Bear City,361.0,...,"[{'Name': 'San Bernardino National Forest', 'A...",[],12,[],NaN,NaN,[],NaN,NaN,NaN
2,Plunge Fire,True,/Date(1150731480000)/,/Date(1138019520000)/,San Bernardino National Forest,NaN,NaN,['San Bernardino'],2-3 miles south of Running Spring,485.0,...,"[{'Name': 'San Bernardino National Forest', 'A...",[],12,[],NaN,NaN,[],NaN,NaN,NaN
3,Perkins Fire,True,/Date(1151262000000)/,/Date(1150730280000)/,Los Padres National Forest,NaN,NaN,['Santa Barbara'],South of the community of New,15043.0,...,"[{'Name': 'Los Padres National Forest', 'Agenc...",[],12,[],NaN,NaN,[],NaN,NaN,NaN
4,Del Puerto Fire,True,/Date(1059030000000)/,/Date(1058695680000)/,CDF Santa Clara Unit,NaN,NaN,['Stanislaus'],Del Puerto Canyon Road/MP 16,5909.0,...,"[{'Name': 'CDF Santa Clara Unit', 'AgencyURL':...",[],12,[],NaN,NaN,[],NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4008,Butter Fire,False,/Date(1756241505340)/,/Date(1756207966000)/,Bureau of Land Management - California Desert ...,NaN,NaN,['Kern'],"Butterbredt Canyon Road, East of Pacific Crest...",48.0,...,[],[],12,[],NaN,NaN,[],NaN,NaN,NaN
4009,Grade Fire,False,/Date(1756275093127)/,/Date(1756218897000)/,CAL FIRE Siskiyou Unit,NaN,NaN,['Siskiyou'],"West Lava Rim Road, Pondosa",10.0,...,[],[],12,[],NaN,NaN,[],NaN,NaN,NaN
4010,3-27 Fire,False,/Date(1756245737990)/,/Date(1756221842000)/,CAL FIRE Lassen-Modoc Unit,NaN,NaN,['Lassen'],"Southeast of Pumpkin Center, Bieber",24.0,...,[],[],12,[],NaN,NaN,[],NaN,NaN,NaN
4011,H-10 Fire,False,/Date(1756241380080)/,/Date(1756202750000)/,Lassen National Forest,NaN,NaN,['Shasta'],"Forest Road 36N18 & Butte Creek, Hat Creek",10.6,...,[],[],12,[],NaN,NaN,[],NaN,NaN,NaN


### Fire data preprocessing

In [126]:
to_float = ['PercentContained','Longitude','Latitude']
to_date = ['Updated','Started']
fire[to_float] = fire[to_float].astype('float64')
fire['Updated'] = pd.to_datetime(fire['Updated'], errors='coerce')
fire['Started'] = pd.to_datetime(fire['Started'], errors='coerce')
fire.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4013 entries, 0 to 4012
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Name              4013 non-null   object        
 1   Updated           4013 non-null   datetime64[ns]
 2   Started           4013 non-null   datetime64[ns]
 3   AdminUnit         3948 non-null   object        
 4   County            3989 non-null   object        
 5   Location          4009 non-null   object        
 6   AcresBurned       4013 non-null   float64       
 7   PercentContained  4013 non-null   float64       
 8   Longitude         4013 non-null   float64       
 9   Latitude          4013 non-null   float64       
 10  Type              2027 non-null   object        
 11  UniqueId          4013 non-null   object        
 12  Url               4013 non-null   object        
 13  StartedDateOnly   0 non-null      float64       
 14  IsActive          4013 n

### singleFire data preprocessing

In [127]:
# miliseconds -> 현재시각으로 변경
to_date2 = ['Updated', 'Started', 'ExtinguishedDate', 'ExtinguishedDateOnly', 'StartedDateOnly', 'ExpectedContainment']

def date_convert(x):
    if pd.isna(x) or x =="":
        return np.nan
    else:
        return datetime.fromtimestamp(x)

for col in to_date2:
    singleFire[col] = singleFire[col].apply(lambda x: int(x[6:-2])//1000)
    singleFire[col] = singleFire[col].replace(singleFire[singleFire[col] < 0][col].unique(), np.nan)
    singleFire[col] = singleFire[col].apply(date_convert)
singleFire

,Name,Final,Updated,Started,AdminUnit,AdminUnitUrl,County,Counties,Location,AcresBurned,...,Agencies,UnifiedCommandAgencies,mapZoom,AdditionalInfo,IncidentCameras,CameraString,Photos,MapUrl3D,DemographicsReportUrl,AdditionalIncidentMapsUrl
0,Sierra Fire,True,2006-02-13 03:00:00,2006-02-06 13:30:00,USFS/Orange County Fire,NaN,NaN,"['Orange', 'Riverside']",Corona/Orange,10584.0,...,"[{'Name': 'USFS/Orange County Fire', 'AgencyUR...",[],12,[],NaN,NaN,[],NaN,NaN,NaN
1,Arrastre Fire,True,2006-06-10 00:33:00,2006-06-09 21:51:00,San Bernardino National Forest,NaN,NaN,['San Bernardino'],6 miles east of Big Bear City,361.0,...,"[{'Name': 'San Bernardino National Forest', 'A...",[],12,[],NaN,NaN,[],NaN,NaN,NaN
2,Plunge Fire,True,2006-06-20 00:38:00,2006-01-23 21:32:00,San Bernardino National Forest,NaN,NaN,['San Bernardino'],2-3 miles south of Running Spring,485.0,...,"[{'Name': 'San Bernardino National Forest', 'A...",[],12,[],NaN,NaN,[],NaN,NaN,NaN
3,Perkins Fire,True,2006-06-26 04:00:00,2006-06-20 00:18:00,Los Padres National Forest,NaN,NaN,['Santa Barbara'],South of the community of New,15043.0,...,"[{'Name': 'Los Padres National Forest', 'Agenc...",[],12,[],NaN,NaN,[],NaN,NaN,NaN
4,Del Puerto Fire,True,2003-07-24 16:00:00,2003-07-20 19:08:00,CDF Santa Clara Unit,NaN,NaN,['Stanislaus'],Del Puerto Canyon Road/MP 16,5909.0,...,"[{'Name': 'CDF Santa Clara Unit', 'AgencyURL':...",[],12,[],NaN,NaN,[],NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4008,Butter Fire,False,2025-08-27 05:51:45,2025-08-26 20:32:46,Bureau of Land Management - California Desert ...,NaN,NaN,['Kern'],"Butterbredt Canyon Road, East of Pacific Crest...",48.0,...,[],[],12,[],NaN,NaN,[],NaN,NaN,NaN
4009,Grade Fire,False,2025-08-27 15:11:33,2025-08-26 23:34:57,CAL FIRE Siskiyou Unit,NaN,NaN,['Siskiyou'],"West Lava Rim Road, Pondosa",10.0,...,[],[],12,[],NaN,NaN,[],NaN,NaN,NaN
4010,3-27 Fire,False,2025-08-27 07:02:17,2025-08-27 00:24:02,CAL FIRE Lassen-Modoc Unit,NaN,NaN,['Lassen'],"Southeast of Pumpkin Center, Bieber",24.0,...,[],[],12,[],NaN,NaN,[],NaN,NaN,NaN
4011,H-10 Fire,False,2025-08-27 05:49:40,2025-08-26 19:05:50,Lassen National Forest,NaN,NaN,['Shasta'],"Forest Road 36N18 & Butte Creek, Hat Creek",10.6,...,[],[],12,[],NaN,NaN,[],NaN,NaN,NaN


### Problem

- Total number of events and Acres burned

In [128]:
print(f"Total number of events: {fire.shape[0]}")
print(f"Total Acres burned: {sum(fire['AcresBurned'])}")

Total number of events: 4013
Total Acres burned: 16644353.53


- Identify high-risk fires (e.g., Acres Burned > 50000 or active more than 30 days: 
extinguished date - started date)

In [167]:
singleFire[['Started','ExtinguishedDate']].dropna().shape[0]

3264

In [176]:
print("The number of both Started and ExtinguishedDate existed is", singleFire[['Started','ExtinguishedDate']].dropna().shape[0], "among", singleFire.shape[0], '\n')

condition1 = (singleFire['AcresBurned'] > 50000)
condition2 = (singleFire['ExtinguishedDate'] - singleFire['Started']) > pd.Timedelta(days=30)

print("high-risk fires")
print(singleFire[condition1 | condition2]['Name'])

The number of both Started and ExtinguishedDate existed is 3264 among 4013 

high-risk fires
2                 Plunge Fire
12           Sawtooth Complex
17      Heart-Millard Complex
33             Uncles Complex
37            Orleans Complex
                ...          
3508                Shoe Fire
3821               Madre Fire
3823               Green Fire
3831      Marble Complex Fire
3926            Gifford Fire 
Name: Name, Length: 869, dtype: object


- Time series of fire events and Acres burned per month

In [186]:
# Started 기준으로 진행
s = singleFire.__deepcopy__()
s['year'] = s['Started'].dt.year
s['month'] = s['Started'].dt.month

events_per_month = s.groupby(['year', 'month'])['Name'].count()
acresBurned_per_month = s.groupby(['year', 'month'])['AcresBurned'].sum()

print("Time series of fire events")
display(pd.DataFrame(events_per_month))

print("Time series Acres burned per month")
display(pd.DataFrame(acresBurned_per_month))

Time series of fire events


Name
year   month      
2003.0 7.0       1
2006.0 1.0       1
       2.0       1
       6.0       5
       7.0      36
...            ...
2025.0 4.0      15
       5.0      79
       6.0     115
       7.0     108
       8.0      90

[189 rows x 1 columns]

Time series Acres burned per month


AcresBurned
year   month             
2003.0 7.0         5909.0
2006.0 1.0          485.0
       2.0        10584.0
       6.0        25247.0
       7.0       364813.0
...                   ...
2025.0 4.0          654.9
       5.0        13460.8
       6.0        24989.9
       7.0       131836.8
       8.0       168721.6

[189 rows x 1 columns]

- Bonus 1. Cluster fires within a 100 km radius that overlap in their active periods

발생 기간이 겹치는 산불 중 반경 100km 이내의 군집 산불 추출

In [232]:
singleFire.columns

Index(['Name', 'Final', 'Updated', 'Started', 'AdminUnit', 'AdminUnitUrl',
       'County', 'Counties', 'Location', 'AcresBurned', 'PercentContained',
       'ControlStatement', 'AgencyNames', 'Longitude', 'Latitude',
       'MapLongitude', 'MapLatitude', 'Type', 'UniqueId', 'Url', 'ExternalUrl',
       'ExtinguishedDate', 'ExtinguishedDateOnly', 'StartedDateOnly',
       'IsActive', 'IsFeatured', 'CalFireIncident', 'NotificationDesired',
       'LogoAgencies', 'CauseOfFire', 'CommandTeam', 'Status', 'Helicopters',
       'Engines', 'Dozers', 'WaterTenders', 'AirTankers', 'OtherAssigned',
       'PersonnelInvolved', 'CrewsInvolved', 'CivilianFatalities',
       'CivilianInjuries', 'FirefighterInjuries', 'FirefighterFatalities',
       'StructuresDestroyed', 'StructuresDamaged', 'StructuresThreatened',
       'StructuresEvacuated', 'EvacuationZoneMap', 'DamageInspectionMap',
       'SocialMedia', 'ConditionStatement', 'Contacts', 'UpdatesQrCode',
       'ExpectedContainment', 'Evacuatio

- Bonus 2. Calculate the average assessed property value by county and identify the top five fires with the greatest property losses over the past 20 years

In [ ]:
# data collect using openAPI

import requests

url = 'https://www.boe.ca.gov/DataPortal/api/odata/County_Assessed_Property_Values_by_Property_Class_and_County'

headers = {"User-Agent": "Mozilla/5.0"}
response = requests.get(url, headers=headers)
data = response.json()
data

{'@odata.context': 'https://www.boe.ca.gov/DataPortal/api/odata/$metadata#County_Assessed_Property_Values_by_Property_Class_and_County',
 'value': [{'AssessmentYearFrom': 2000,
   'AssessmentYearTo': 2001,
   'County': 'Alameda',
   'LandAssessedValue': 34717087000,
   'ImprovementsAssessedValue': 70021609000,
   'PersonalPropertyAssessedValue': 7274160000,
   'Exemptions': 2885547000,
   'NetTotal': 109127309000,
   'YearToYearChange': 10.0},
  {'AssessmentYearFrom': 2000,
   'AssessmentYearTo': 2001,
   'County': 'Alpine',
   'LandAssessedValue': 101844000,
   'ImprovementsAssessedValue': 165528000,
   'PersonalPropertyAssessedValue': 6247000,
   'Exemptions': 478000,
   'NetTotal': 273141000,
   'YearToYearChange': 11.0},
  {'AssessmentYearFrom': 2000,
   'AssessmentYearTo': 2001,
   'County': 'Amador',
   'LandAssessedValue': 808762000,
   'ImprovementsAssessedValue': 1470829000,
   'PersonalPropertyAssessedValue': 88822000,
   'Exemptions': 55920000,
   'NetTotal': 2312493000,
   

In [212]:
# data = data['value']
cols = data[0].keys()
data_array = np.array([[value for value in data[i].values()] for i in range(len(data))])
assess_value = pd.DataFrame(data_array, columns=cols)

to_int = ['AssessmentYearFrom', 'AssessmentYearTo', 'LandAssessedValue', 'ImprovementsAssessedValue', 'PersonalPropertyAssessedValue',
          'Exemptions', 'NetTotal']
for col in to_int:
    assess_value[col] = assess_value[col].astype(int)
assess_value['YearToYearChange'] = assess_value['YearToYearChange'].astype(float)

assess_value.to_excel(r'C:\Users\SSAFY\vscode_psb\liveCodingTest\assess_value.xlsx', index=False)
display(assess_value)

,AssessmentYearFrom,AssessmentYearTo,County,LandAssessedValue,ImprovementsAssessedValue,PersonalPropertyAssessedValue,Exemptions,NetTotal,YearToYearChange
0,2000,2001,Alameda,34717087000,70021609000,7274160000,2885547000,109127309000,10.0
1,2000,2001,Alpine,101844000,165528000,6247000,478000,273141000,11.0
2,2000,2001,Amador,808762000,1470829000,88822000,55920000,2312493000,5.7
3,2000,2001,Butte,3523141000,6326376000,621729000,390854000,10080393000,5.3
4,2000,2001,Calaveras,1105378000,2079017000,73226000,24224000,3233397000,6.2
...,...,...,...,...,...,...,...,...,...
1446,2024,2025,Tulare,12943257000,35892662000,2774381000,1416897000,50193403000,8.4
1447,2024,2025,Tuolumne,2873980000,6807448000,367535000,336373000,9712590000,4.8
1448,2024,2025,Ventura,91256113000,88647860000,5437273000,5252939000,180088307000,4.3
1449,2024,2025,Yolo,11526486000,27668861000,1307208000,2311724000,38190830000,5.1


NetTotal contains land, improvedments(building... etc), personal property also excepts Exemptions.

**NetTotal = LandAssessedValue + ImprovementsAssessedValue + PersonalPropertyAssessedValue - Exemptions**

**Property loss = NetTotal * (StructuresDestroyed + StructuresDamaged)**

In [ ]:
avg_value = assess_value.groupby('County')['NetTotal'].mean()
# print("Average NetTotal by county")
# display(pd.DataFrame(avg_value))
print('Top 5 property loss fire')

Average NetTotal by county


,NetTotal
County,
Alameda,2.340267e+11
Alpine,6.641381e+08
Amador,4.477748e+09
Butte,1.872456e+10
Calaveras,6.387267e+09
Colusa,2.759149e+09
Contra Costa,1.673523e+11
Del Norte,1.708680e+09
El Dorado,2.787463e+10


### Data Science Track

- Display charts (matplotlib/seaborn/Plotly)

In [231]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import streamlit as st

- Show alert dashboard with filtering (필터링 기능이 있는 경보 데쉬보드)

- Interactive map using *folium* or *plotly*

- Use cross filtering option where when you filter something in the table, then the map will filter out those points in a similar way.